#### Re-ranking Implementation

In [7]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser

In [8]:
### load text file
loader = TextLoader('langchain_sample.txt')
raw_documents = loader.load()
raw_documents

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\nLangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\nRetrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional sparse retrieval method that scores documents based on keyword matching. Although fast, it often struggles with synonyms and semantic similari

In [9]:
## split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,chunk_overlap=50
)
docs = splitter.split_documents(raw_documents)
docs

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [10]:
# user query
query = "how can i use langchain to build an application with memory and tools?"

In [11]:
## FAISS and HuggingFace Embeddings

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model='all-miniLM-L6-v2'
)
vector_store = FAISS.from_documents(docs,embedding_model)
retriever = vector_store.as_retriever(serch_kwargs={'k':3})

/home/bhupen/Python_Learning/RAG/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import os
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [13]:
from langchain.chat_models import init_chat_model

llm = init_chat_model('groq:gemma2-9b-it')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f1b0e535be0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f1b0e5367b0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [14]:
# prompt template
prompt = PromptTemplate.from_template("""
You are a helpful assistant.Your task is to rank the following documents from most to least relevant to the user's question.
                        User Question: "{question}"
                        Documents: {documents}

                        Instructions:
                            - Think about the relevance of each document to the user's question.
                            - Return a list of document indices in ranked order,starting from the most relevant.
                            
                        Ouput format : comma-separated document indices (eg.2,1,3,0...)
                                      
""")

In [15]:
retrieved_docs = retriever.invoke(query)
print(retrieved_docs)

[Document(id='bc4b0709-b3db-434d-9194-1bbc27c91929', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'), Document(id='07824dd5-1a83-4486-9eac-19ab1a4c5629', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'), Document(id='b85415d8-982d-41e8-82b0-28316c79f155', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Fac

In [17]:
chain = prompt | llm | StrOutputParser()

In [18]:
doc_lines = [f'{i+1}. {doc.page_content}' for i,doc in enumerate(retrieved_docs)]
format_docs = "\n".join(doc_lines)

In [19]:
doc_lines

['1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.',
 '2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.',
 '3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.',
 '4. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and comp

In [20]:
format_docs

'1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.\n2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\n3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\n4. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed ind

In [24]:
response = chain.invoke({"question":query,"documents":format_docs})
response

'2,1,4,3 \n'

In [25]:
indices = [int(x.strip()) - 1 for x in response.split(',') if x.strip().isdigit()]
indices

[1, 0, 3, 2]

In [26]:
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]

In [28]:
for i,doc in enumerate(reranked_docs,1):
    print(f"Rank:{i}: {doc.page_content}")

Rank:1: LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.
Rank:2: LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.
Rank:3: FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with search and code execution.